**this code from : https://towardsdatascience.com/question-answering-with-a-fine-tuned-bert-bc4dafd45626**

In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Data Loading from Local CSV File

In [188]:
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

#put train.csv file in this notebook
data = pd.read_csv("train.csv")
#pre processing
del data["id"]
del data["title"]

#remeber to convert context in train.csv to text 

print ( data.head() )
print ( data["text"]  )


                                                text  \
0  جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...   
1  جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...   
2  جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...   
3  جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...   
4  جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...   

                                          question  \
0                   - من هو جمال أحمد حمزة خاشقجي؟   
1        - متى ولد جمال أحمد حمزة خاشقجي وتوفي؟ ال   
2      - في أي مدينة ولد جمال أحمد حمزة خاشقجي؟ ال   
3   - في أي صحيفة قام بكتابة عمود منذ عام 2017؟ ال   
4  - كيف وصفها في الصحف ووسائل الإعلام الدولية؟ ال   

                                             answers  
0  {'text': array(['صحفي وإعلامي'], dtype=object)...  
1  {'text': array(['حمزة خاشقجي (13 أكتوبر 1958، ...  
2  {'text': array(['المدينة المنورة'], dtype=obje...  
3  {'text': array(['واشنطن بوست'], dtype=object),...  
4  {'text': array(['وُصف في الصحف وأجهزة الاعلام ...  
0      ج

In [189]:
print("Number of question and answers: ", len(data))

Number of question and answers:  693


Building the Chatbot
The best part about using these pre-trained models is that you can load the model and its tokenizer in just two simple lines of code. 😲 Isn’t it simply wow? For tasks like text classification, we need to fine-tune BERT on our dataset. But for question answering tasks, we can even use the already trained model and get decent results even when our text is from a completely different domain. To get decent results, we are using a BERT model which is fine-tuned on the SQuAD benchmark.

For our task, we will use the BertForQuestionAnswering class from the transformers library.

In [234]:
model = BertForQuestionAnswering.from_pretrained('asafaya/bert-base-arabic')
tokenizer = BertTokenizer.from_pretrained('asafaya/bert-base-arabic')

Some weights of the model checkpoint at asafaya/bert-base-arabic were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at asafaya/bert-base-arabic and are newly i

Asking a Question
Let’s randomly select a question number.

In [244]:
random_num = np.random.randint(0,len(data))
question = data["question"][random_num]
text = data["text"][random_num]



#question = data["question"][1]
#text = data["text"][1]

print("question : " +question )
print("text : " +text )

#question = " ما اسم كلية ياسمين ؟ "
#text = "ياسمين لديها اخ واحد اسمه محمد عادل و ياسمين في كلية الحاسبات و الذكاء الاصطناعي لديها العديد من الاصدقاء "

question : ماهي المهنة التي كان يمارسها الفينيقيون؟
text : في القدم، سكن الفينيقيون أرض لبنان الحالية مع جزء من أرض سوريا وفلسطين، وهؤلاء قوم ساميون اتخذوا من الملاحة والتجارة مهنة لهم، وازدهرت حضارتهم طيلة 2500 سنة تقريبًا (من حوالي سنة 3000 حتى سنة 539 ق.م). وقد مرّت على لبنان عدّة حضارات وشعوب استقرت فيه منذ عهد الفينيقين، مثل المصريين القدماء، الآشوريين، الفرس، الإغريق، الرومان، الروم البيزنطيين، العرب، الصليبيين، الأتراك العثمانيين، فالفرنسيين.


Let’s tokenize the question and text as a pair.

In [245]:
input_ids = tokenizer.encode(question, text)
print("The input has a total of {} tokens.".format(len(input_ids)))
print ( input_ids )

The input has a total of 116 tokens.
[2, 17490, 18372, 1833, 1841, 12339, 1731, 23547, 14222, 1733, 221, 3, 1725, 3955, 219, 7088, 23547, 14222, 1733, 3648, 2851, 7252, 1776, 4412, 1726, 3648, 3468, 24067, 219, 18637, 6954, 9801, 1733, 8795, 1751, 1726, 14002, 12923, 16611, 2819, 219, 9254, 2986, 1920, 5893, 21923, 14338, 8361, 1034, 2634, 6527, 12, 1726, 5543, 2634, 22423, 2123, 2634, 7847, 1039, 245, 18, 248, 13, 18, 2269, 3484, 1747, 2851, 3630, 5893, 1912, 12536, 1870, 22122, 1013, 2229, 2597, 5609, 23547, 14222, 1017, 219, 2358, 8916, 25946, 219, 2324, 24130, 219, 17551, 219, 3488, 2028, 219, 21225, 219, 7903, 2062, 3573, 28683, 219, 3153, 219, 17084, 2696, 219, 20536, 14016, 1724, 219, 22314, 1918, 16084, 18, 3]


Let’s see how many tokens this question and text pair have.

To look at what our tokenizer is doing, let’s just print out the tokens and their IDs.

In [246]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
for token, id in zip(tokens, input_ids):
    print('{:8}{:8,}'.format(token,id))

[CLS]          2
ماهي      17,490
المهنة    18,372
التي       1,833
كان        1,841
يمارس     12,339
##ها       1,731
الفين     23,547
##يقي     14,222
##ون       1,733
؟            221
[SEP]          3
في         1,725
القدم      3,955
،            219
سكن        7,088
الفين     23,547
##يقي     14,222
##ون       1,733
ارض        3,648
لبنان      2,851
الحالية    7,252
مع         1,776
جزء        4,412
من         1,726
ارض        3,648
سوريا      3,468
وفلسطين   24,067
،            219
وهولاء    18,637
قوم        6,954
سامي       9,801
##ون       1,733
اتخذ       8,795
##وا       1,751
من         1,726
الملاحة   14,002
والتجارة  12,923
مهنة      16,611
لهم        2,819
،            219
واز        9,254
##ده       2,986
##رت       1,920
حض         5,893
##ارتهم   21,923
طيلة      14,338
250        8,361
##0        1,034
سنة        2,634
تقريبا     6,527
(             12
من         1,726
حوالي      5,543
سنة        2,634
3000      22,423
حتى        2,123
سنة        2,634
53         7,8

In [247]:
#first occurence of [SEP] token
sep_idx = input_ids.index(tokenizer.sep_token_id)
print("SEP token index: ", sep_idx)
#number of tokens in segment A (question) - this will be one more than the sep_idx as the index in Python starts from 0
num_seg_a = sep_idx+1
print("Number of tokens in segment A: ", num_seg_a)
#number of tokens in segment B (text)
num_seg_b = len(input_ids) - num_seg_a
print("Number of tokens in segment B: ", num_seg_b)
#creating the segment ids
segment_ids = [0]*num_seg_a + [1]*num_seg_b
print( [1]*num_seg_b )
print ( [0]*num_seg_a )
print ( segment_ids)
#Segment embeddings help BERT in differentiating a question from the text. zero question , one answer
#making sure that every input token has a segment id
assert len(segment_ids) == len(input_ids)

SEP token index:  11
Number of tokens in segment A:  12
Number of tokens in segment B:  104
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


Let’s now feed this to our model.

In [248]:
#token input_ids to represent the input and token segment_ids to differentiate our segments - question and text
output = model(torch.tensor([input_ids]),  token_type_ids=torch.tensor([segment_ids]))

Looking at the most probable start and end words and providing answers only if the end token is after the start token


In [249]:
#tokens with highest start and end scores
answer_start = torch.argmax(output.start_logits)
answer_end = torch.argmax(output.end_logits)
if answer_end >= answer_start:
    answer = " ".join(tokens[answer_start:answer_end+1])
else:
    print("I am unable to find the answer to this question. Can you please ask another question?")
    
print("\nQuestion:\n{}".format(question.capitalize()))
print("\nAnswer:\n{}.".format(answer.capitalize()))


Question:
ماهي المهنة التي كان يمارسها الفينيقيون؟

Answer:
في القدم ، سكن الفين ##يقي ##ون ارض لبنان الحالية مع جزء من ارض سوريا وفلسطين ، وهولاء قوم سامي ##ون اتخذ ##وا من الملاحة والتجارة مهنة لهم ، واز ##ده ##رت حض ##ارتهم.


In [250]:
answer = tokens[answer_start]
for i in range(answer_start+1, answer_end+1):
    if tokens[i][0:2] == "##":
        answer += tokens[i][2:]
    else:
        answer += " " + tokens[i]
        
print("\nQuestion:\n{}".format(question.capitalize()))
print ("answer " + answer )


Question:
ماهي المهنة التي كان يمارسها الفينيقيون؟
answer في القدم ، سكن الفينيقيون ارض لبنان الحالية مع جزء من ارض سوريا وفلسطين ، وهولاء قوم ساميون اتخذوا من الملاحة والتجارة مهنة لهم ، وازدهرت حضارتهم


r, “Karingu” is a rare word so wordpiece split it into the words, “Karin” and “##gu”.

##SUFFIX’ (if any suffix at all — for example, “run”, “##ning”, “##ner”).